In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV

In [0]:
from warnings import filterwarnings
filterwarnings("ignore")

In [0]:
downloaded = drive.CreateFile({'id':"1Tt8jUV-JGbtkaB6dcrqMRmQPTwAReRAj"}) 
downloaded.GetContentFile('kc_house_data.csv') 

In [0]:
df = pd.read_csv('kc_house_data.csv')

In [0]:
df = df.dropna()

In [8]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [0]:
df.drop(["id","date"], axis = 1, inplace = True)

In [0]:
y = df.price.values.reshape(-1,1)
x = df.drop(["price"], axis = 1)

In [0]:
scaler = MinMaxScaler()
x = scaler.fit_transform(x)
y = scaler.fit_transform(y)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y , test_size = 0.2, random_state = 0, shuffle=True)

In [13]:
gbm_model = XGBRegressor().fit(x_train, y_train)

[15:43:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [14]:
y_pred = gbm_model.predict(x_test)
r2_score(y_test, y_pred)

0.8730359233442346

In [0]:
?gbm_model

In [0]:
gbm_params = {"n_estimators" : [ 500, 600, 750, 1000],
              "learning_rate" : [0.1, 0.15, 0.05],
              "max_depth" : [6, 5, 4],
              "n_jobs" : [0.5, 1, 2, 3]
              }

In [0]:
gbm_cv_model = GridSearchCV(gbm_model, 
                            gbm_params, 
                            cv=10, 
                            n_jobs=-1, 
                            verbose=2).fit(x_train, y_train)

Fitting 10 folds for each of 144 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 39.9min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed: 76.8min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed: 139.1min
[Parallel(n_jobs=-1)]: Done 1009 tasks      | elapsed: 206.1min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed: 298.1min finished


[14:13:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [0]:
gbm_cv_model.best_params_

In [0]:
gbm_tuned = XGBRegressor( learning_rate = 0.1,
                          max_depth = 4,
                          n_estimators = 1000,
                          n_jobs = 1,
                          objective ='reg:squarederror').fit(x_train, y_train)

In [18]:
y_pred = gbm_tuned.predict(x_test)
r2_score(y_test, y_pred)

0.9015050283654841